# Code 1 - 49조

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import time
import openpyxl
import pandas as pd

In [8]:
pip install webdriver_manager

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


### 엑셀 파일 생성하기
파일명, 시트명, 컬럼명을 정해줍니다.

In [37]:
#크롤링 데이터 저장할 리스트 생성
ws = []         
ws.append(['브랜드','상품명','카테고리','정가','할인가','아이디','리뷰','별점','피부정보','피부타입','피부고민','자극도'])  #컬럼명 제공
ws

[['브랜드',
  '상품명',
  '카테고리',
  '정가',
  '할인가',
  '아이디',
  '리뷰',
  '별점',
  '피부정보',
  '피부타입',
  '피부고민',
  '자극도']]

In [9]:
######################제공하는 코드 건드리지 마세요 #############################

#크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

#브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option('detach',True)

#불필요한 에러 메세지 없애기
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

### 올리브영 스킨케어 랭킹 링크를 main_url로 두고 10위까지 제품 상세 링크를 sub_url에 저장하기

In [10]:
main_url = 'https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010001&pageIdx=1&rowsPerPage=8'

response = requests.get(main_url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

# CSS 선택자를 수정하여 올바른 요소를 대상으로 선택합니다.
links = soup.select("#Container > div.best-area > div.TabsConts.on > ul > li > div > a")

sub_url = []

# enumerate 함수를 사용하여 rank를 자동으로 처리
for rank, link in enumerate(links, 1):
    sub_url.append(link['href'])
    if rank >= 10:
        break

print(sub_url)

['https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000211001&dispCatNo=90000010009&trackingCd=Best_Sellingbest&t_page=랭킹&t_click=판매랭킹_스킨케어_상품상세&t_number=1', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000211089&dispCatNo=90000010009&trackingCd=Best_Sellingbest&t_page=랭킹&t_click=판매랭킹_스킨케어_상품상세&t_number=2', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000205085&dispCatNo=90000010009&trackingCd=Best_Sellingbest&t_page=랭킹&t_click=판매랭킹_스킨케어_상품상세&t_number=3', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000184924&dispCatNo=90000010009&trackingCd=Best_Sellingbest&t_page=랭킹&t_click=판매랭킹_스킨케어_상품상세&t_number=4', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000211401&dispCatNo=90000010009&trackingCd=Best_Sellingbest&t_page=랭킹&t_click=판매랭킹_스킨케어_상품상세&t_number=5', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000179685&dispCatNo=90000010009&track

### 크롤링 함수 만들기
제품 상세 페이지에서 상단 제품정보(브랜드명,상품명,카테고리,정가,할인가)와  
하단의 리뷰고객 좌픅정보(아이디,별점,피부정보)와 우측정보(피부타입,피부고민,자극도)를 가져옵니다.  
위의 데이터를 엑셀에 계속 추가하는 함수를 만듭니다. 

In [11]:
def customer_info():
    
    soup = BeautifulSoup(driver.page_source, "html.parser")

    for j in range(0,10,1):

        #브랜드
        try:
            brand = driver.find_element(By.ID, 'onlBrndNm').get_attribute('value')
        except:
            brand ="없음"

        #상품명
        try:
            p_name = driver.find_element(By.ID, 'goodsNm').get_attribute('value')
        except:
            p_name="없음"

        #카테고리
        try:
            p_category = driver.find_element(By.ID, 'eigeneSmlDispName').get_attribute('value')
        except:
            p_category="없음"

        #정가
        try:
            price = soup.select_one("#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-1 > strike").text.strip()
        except:
            price=0

        #할인가
        try:            
            discount = soup.select_one('#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-2 > strong').text.strip() 
        except:
            discount=0    
            
        #고객 아이디
        try:
            _id=soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.info > div > p.info_user > a.id').text.strip()
        except:
            _id ="없음"        
        #리뷰
        try:
            _id=soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.review_cont > div > div.txt_inner > a.id').text.strip()
        except:
            _id ="없음"

        #별점
        try:
            _star = soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.score_area > span.review_point > span').text.strip() 
        except:
            _star="없음"

        #고객 피부 정보
        try:
            _info = soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.info > div > p.tag').text.strip() 
        except:
            _info = "없음"


        #피부 타입
        try:
            skin_type = soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span').text.strip()
        except:
            skin_type = "없음"
            
        #피부 고민
        try:
            skin_trouble = soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.poll_sample > dl:nth-child(2) > dd > span').text.strip()
        except:
            skin_trouble = "없음"
        #자극도
        try:
            skin_irritation = soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.poll_sample > dl:nth-child(3) > dd > span').text.strip()
        except:
            skin_irritation = "없음"
            
        #엑셀 데이터 쌓기
        ws.append([brand,p_name,p_category,price,discount,_id, _star, _info, skin_type, skin_trouble, skin_irritation])
        time.sleep(1)

### 웹페이지 해당 주소로 이동하여 크롤링함수를 실행하는 반복문을 만듭니다.
반복문에 들어가야 하는 것  
1. 리뷰버튼 클릭  
2. 화면 맨 아래까지 스크롤하기 (코드제공)
3. 10페이지 이하일 때 : 10페이지 크롤링 한 후 다음페이지 화살표 버튼 누르기 , 마지막 페이지인 경우 last_page==True
4. 11페이지부터 규칙을찾아 다음페이지로 이동하게 만들기, 마지막 페이지인 경우 last_page==True

In [12]:
#웹페이지 해당 주소 이동
for i in range(0,10):          #전체 제품을 한번에 크롤링하지 않고 나눠서 크롤링 할 경우 sub_url 인덱스 고려해서 range숫자 변경

    driver = webdriver.Chrome()  # 웹 드라이버 재시작
    driver.implicitly_wait(5)  #웹페이지 로딩 될때까지 5초는 기다림
    driver.maximize_window()   #화면 최대화
    driver.get(sub_url[i])          
    time.sleep(3)


    #리뷰버튼 클릭
    rv = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a') #셀레니움 최신버전을 사용하는 관계로 해당 코드로 수정했습니다
    #rv = driver.find_element_by_xpath('//*[@id="reviewInfo"]/a')
    rv.click()
    time.sleep(3)
    

    #리뷰 하단 끝까지 스크롤하는 함수 (빈칸없음.그대로 코드 사용가능)
    before_h = driver.execute_script("return window.scrollY")        #스크롤 전 높이
    
    #화면 맨아래까지 스크롤
    while True:
        driver.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)

        time.sleep(1)

        #스크롤 후 높이
        after_h = driver.execute_script("return window.scrollY")

        #스크롤 값이 같으면 스크롤 멈춤
        if after_h == before_h:
            break
        before_h = after_h           
        
        
    last_page=False
    
    for k in range(1,101):  #100페이지 크롤링 한다고 했을 때 (상품당 최대 100페이지까지 있음)

        #마지막 페이지면 종료
        if last_page == True:
            break
        
        #10페이지 이하일 때와 11페이지 이상일 때 각각 코드를 작성하지 않고 k의 변화에 따라 move를 변화시켜 한번에 처리함
        #11페이지 이상인 경우 move는 2~11이 반복됨
        if k<11:
            move=k
        else:
            move=(k%10)+1
            if move==1:
                move=11

        try:
            customer_info()
            pg = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{move}]').click()      
            #pg = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{move}]').click() #일부 제품은 페이지 넘기기가 dif[7]에 있음
            

        except:
            last_page = True
                   

driver.quit()

### 크롤링한 결과를 엑셀에 저장 (상단에서 만든 엑셀 파일명과 동일하게)

In [13]:
columns = ['브랜드', '상품명', '카테고리', '정가', '할인가', '아이디', '별점', '피부정보', '피부타입', '피부고민', '자극도']
df = pd.DataFrame(ws[1:], columns=columns)
df.head()

,브랜드,상품명,카테고리,정가,할인가,아이디,별점,피부정보,피부타입,피부고민,자극도
0,라네즈,[10월 올영픽][JIN 미니포토북 선착순 증정] 라네즈 크림스킨 170ml 기획 ...,스킨/토너,"33,000","23,760",려워니진,5점만점에 5점,없음,건성에 좋아요,보습에 좋아요,자극없이 순해요
1,라네즈,[10월 올영픽][JIN 미니포토북 선착순 증정] 라네즈 크림스킨 170ml 기획 ...,스킨/토너,"33,000","23,760",rose7****,5점만점에 5점,없음,복합성에 좋아요,보습에 좋아요,자극없이 순해요
2,라네즈,[10월 올영픽][JIN 미니포토북 선착순 증정] 라네즈 크림스킨 170ml 기획 ...,스킨/토너,"33,000","23,760",별향,5점만점에 5점,건성 쿨톤 각질 다크서클,건성에 좋아요,보습에 좋아요,자극없이 순해요
3,라네즈,[10월 올영픽][JIN 미니포토북 선착순 증정] 라네즈 크림스킨 170ml 기획 ...,스킨/토너,"33,000","23,760",yop,5점만점에 5점,복합성 쿨톤 모공 블랙헤드,건성에 좋아요,보습에 좋아요,자극없이 순해요
4,라네즈,[10월 올영픽][JIN 미니포토북 선착순 증정] 라네즈 크림스킨 170ml 기획 ...,스킨/토너,"33,000","23,760",yop,5점만점에 5점,복합성 쿨톤 모공 블랙헤드,건성에 좋아요,보습에 좋아요,자극없이 순해요


In [14]:
df.to_excel('Top Rank 10.xlsx', index=False)

In [23]:
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 불러오기 (크롤링한 데이터)
df = pd.read_excel('Top Rank 10.xlsx')

# 별점이 없는 데이터를 제외
df = df[df['별점'] != '없음']

# 별점에서 숫자만 추출 (예: "5점만점에 5점"에서 숫자만 추출)
df['별점'] = df['별점'].apply(lambda x: re.findall(r'\d+', x)[0])  # 숫자만 추출
df['별점'] = pd.to_numeric(df['별점'])  # 숫자형으로 변환

# 사용자-아이템 평점 행렬 만들기 (피벗 테이블)
pivot_table = df.pivot_table(index='아이디', columns='상품명', values='별점')

# 결측값을 0으로 채워서 사용자가 평가하지 않은 제품은 0으로 처리
pivot_table = pivot_table.fillna(0)

# 코사인 유사도를 통해 아이템 간 유사도 계산
item_similarity = cosine_similarity(pivot_table.T)  # 아이템 간 유사도 계산
item_similarity_df = pd.DataFrame(item_similarity, index=pivot_table.columns, columns=pivot_table.columns)

# 유사도가 높은 상위 n개의 아이템 추천 (예: 특정 상품에 대한 추천)
def get_similar_items(product_name, n=5):
    if product_name not in item_similarity_df.index:
        print(f"'{product_name}'은(는) 목록에 없는 상품명입니다.")
        return None
    similar_items = item_similarity_df[product_name].sort_values(ascending=False)[1:n+1]
    return similar_items

# 특정 상품을 기반으로 추천받기
product_name = '[10월 올영픽]  아누아 피디알엔 히알루론산 캡슐 100 세럼 30ml 기획 (+눈물세럼 1ml*10ea)'  # 추천 기준이 될 상품명 입력
recommended_items = get_similar_items(product_name)

# 추천 결과 출력
if recommended_items is not None:
    print(f"'{product_name}'와 유사한 상위 {len(recommended_items)}개 제품:")
    print(recommended_items)

'[10월 올영픽]  아누아 피디알엔 히알루론산 캡슐 100 세럼 30ml 기획 (+눈물세럼 1ml*10ea)'와 유사한 상위 5개 제품:
상품명
[10월 올영픽][모공/탄력/에센스]아이오페 레티놀 슈퍼 바운스 세럼 30ml 기획 (+20ml+선 프로텍터25ml)     0.263053
[10월 올영픽/Lee콜라보] 이니스프리 레티놀 시카 흔적 앰플 30ml 기획                           0.051084
에스트라 아토베리어365 크림 80ml 기획 (+하이드로에센스 25ml+리제덤세럼 7ml)                    0.018152
[10월 올영픽][JIN 미니포토북 선착순 증정] 라네즈 크림스킨 170ml 기획 (+130ml 리필+미스트펌프)       0.014857
[10월 올영픽/한정기획] 웰라쥬 리얼 히알루로닉 블루 100 앰플 75ml 1+1 기획 (+크림10ml+마스크1매)     0.012306
Name: [10월 올영픽]  아누아 피디알엔 히알루론산 캡슐 100 세럼 30ml 기획 (+눈물세럼 1ml*10ea), dtype: float64


In [31]:
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# 데이터 불러오기 (크롤링한 데이터)
df = pd.read_excel('Top Rank 10.xlsx')

# 별점이 없는 데이터를 제외
df = df[df['별점'] != '없음']

# 별점에서 숫자만 추출 (예: "5점만점에 5점"에서 숫자만 추출)
df['별점'] = df['별점'].apply(lambda x: re.findall(r'\d+', x)[0])  # 숫자만 추출
df['별점'] = pd.to_numeric(df['별점'])  # 숫자형으로 변환

# 사용자-아이템 평점 행렬 만들기 (피벗 테이블)
pivot_table = df.pivot_table(index='아이디', columns='상품명', values='별점')

# 결측값을 0으로 채워서 사용자가 평가하지 않은 제품은 0으로 처리
pivot_table = pivot_table.fillna(0)

# 아이템 간 유사도 계산 (코사인 유사도 사용)
item_similarity = cosine_similarity(pivot_table.T)  # 아이템 간 유사도 계산
item_similarity_df = pd.DataFrame(item_similarity, index=pivot_table.columns, columns=pivot_table.columns)

# 특정 상품에 대한 평점 예측 함수
def predict_ratings_for_item(product_name, pivot_table, item_similarity_df):
    if product_name not in item_similarity_df.index:
        print(f"'{product_name}'은(는) 목록에 없는 상품명입니다.")
        return None

    # 해당 상품과 유사한 상품들의 유사도 가져오기
    similar_items = item_similarity_df[product_name]
    
    # 각 사용자의 상품 평점 가져오기
    product_ratings = pivot_table[product_name]
    
    # 유사한 상품들의 평점을 곱해서 예측 평점 계산
    weighted_ratings = pivot_table.dot(similar_items)  # 유사도 * 평점
    sum_of_similarities = np.abs(similar_items).sum()
    
    # 예측 평점 계산 (가중치 적용한 평균)
    predicted_ratings = weighted_ratings / sum_of_similarities

    return pd.DataFrame(predicted_ratings, columns=['예측 평점']).sort_values(by='예측 평점', ascending=False)

# 평가지표 계산 함수 (RMSE, MAE)
def evaluate_model(actual_ratings, predicted_ratings):
    # 실제 평점과 예측된 평점의 차이를 계산할 때 결측값(평가하지 않은 항목)을 제거
    actual = actual_ratings[actual_ratings > 0].values
    predicted = predicted_ratings[actual_ratings > 0].values

    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    
    # MAE 계산
    mae = mean_absolute_error(actual, predicted)
    
    return rmse, mae

# 예시: 특정 상품에 대한 평점 예측 및 평가
product_name = '[10월 올영픽]  아누아 피디알엔 히알루론산 캡슐 100 세럼 30ml 기획 (+눈물세럼 1ml*10ea)'  # 예측할 상품명

# 예측 평점
predicted_ratings_df = predict_ratings_for_item(product_name, pivot_table, item_similarity_df)

# 실제 평점과 예측된 평점 비교 및 평가
if predicted_ratings_df is not None:
    actual_ratings = pivot_table[product_name]  # 실제 평점 데이터
    predicted_ratings = predicted_ratings_df['예측 평점']  # 예측된 평점
    
    # 모델 평가 (RMSE, MAE)
    rmse, mae = evaluate_model(actual_ratings, predicted_ratings)
    
    # 결과 출력
    print(f"'{product_name}'의 예측된 평점:")
    print(predicted_ratings_df.head(10))  # 상위 10개의 예측된 평점 출력
    
    print(f"\n평가지표:")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")

'[10월 올영픽]  아누아 피디알엔 히알루론산 캡슐 100 세럼 30ml 기획 (+눈물세럼 1ml*10ea)'의 예측된 평점:
                  예측 평점
아이디                    
하니해코덕해         4.807081
m튤립            4.770703
하쿠나마타타고래꼬리     4.770703
달고달고달디단        4.770703
Camellialatte  4.705084
월급지켜           4.651149
치키차카초초         4.651149
유댕좌임           4.651149
후루리            4.639186
삐빠뽀뿌           4.639186

평가지표:
RMSE: 1.208309179730696
MAE: 1.1367665903802249


In [33]:
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# 데이터 불러오기 (크롤링한 데이터)
df = pd.read_excel('Top Rank 10.xlsx')

# 별점이 없는 데이터를 제외
df = df[df['별점'] != '없음']

# 별점에서 숫자만 추출 (예: "5점만점에 5점"에서 숫자만 추출)
df['별점'] = df['별점'].apply(lambda x: re.findall(r'\d+', x)[0])  # 숫자만 추출
df['별점'] = pd.to_numeric(df['별점'])  # 숫자형으로 변환

# 사용자-아이템 평점 행렬 만들기 (피벗 테이블)
pivot_table = df.pivot_table(index='아이디', columns='상품명', values='별점')

# 결측값을 0으로 채워서 사용자가 평가하지 않은 제품은 0으로 처리
pivot_table = pivot_table.fillna(0)

# 아이템 간 유사도 계산 (코사인 유사도 사용)
item_similarity = cosine_similarity(pivot_table.T)  # 아이템 간 유사도 계산
item_similarity_df = pd.DataFrame(item_similarity, index=pivot_table.columns, columns=pivot_table.columns)

# 겹치는 사용자 추출 함수
def get_common_users(product_name, pivot_table):
    # 특정 상품을 평가한 사용자들
    users_who_rated_product = pivot_table[product_name][pivot_table[product_name] > 0].index
    
    # 다른 상품을 평가한 사용자들
    other_rated_products = pivot_table.drop(columns=[product_name])
    users_who_rated_others = other_rated_products[other_rated_products > 0].stack().reset_index()['아이디'].unique()
    
    # 겹치는 사용자 추출
    common_users = set(users_who_rated_product).intersection(users_who_rated_others)
    
    return list(common_users)

# 특정 상품에 대한 평점 예측 함수 (겹치는 사용자 10명 대상)
def predict_ratings_for_item_with_common_users(product_name, pivot_table, item_similarity_df):
    # 겹치는 사용자 추출
    common_users = get_common_users(product_name, pivot_table)
    
    # 겹치는 사용자 중 10명을 랜덤으로 추출
    if len(common_users) > 10:
        sampled_users = np.random.choice(common_users, 10, replace=False)
    else:
        sampled_users = common_users
    
    # 특정 상품의 예측 평점 계산
    similar_items = item_similarity_df[product_name]
    weighted_ratings = pivot_table.dot(similar_items)
    sum_of_similarities = np.abs(similar_items).sum()
    predicted_ratings = weighted_ratings / sum_of_similarities
    
    # 예측 평점을 겹치는 사용자들만 필터링
    predicted_ratings_for_common_users = predicted_ratings.loc[sampled_users]
    
    return pd.DataFrame(predicted_ratings_for_common_users, columns=['예측 평점']).sort_values(by='예측 평점', ascending=False), sampled_users

# 평가지표 계산 함수 (RMSE, MAE)
def evaluate_model(actual_ratings, predicted_ratings):
    # 실제 평점과 예측된 평점의 차이를 계산할 때 결측값(평가하지 않은 항목)을 제거
    actual = actual_ratings[actual_ratings > 0].values
    predicted = predicted_ratings[actual_ratings > 0].values

    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    
    # MAE 계산
    mae = mean_absolute_error(actual, predicted)
    
    return rmse, mae

# 예시: 특정 상품에 대한 평점 예측 (겹치는 사용자 10명 대상으로)
product_name = '[10월 올영픽]  아누아 피디알엔 히알루론산 캡슐 100 세럼 30ml 기획 (+눈물세럼 1ml*10ea)'  # 예측할 상품명

# 예측 평점
predicted_ratings_df, sampled_users = predict_ratings_for_item_with_common_users(product_name, pivot_table, item_similarity_df)

# 실제 평점과 예측된 평점 비교 및 평가
if predicted_ratings_df is not None and not predicted_ratings_df.empty:
    actual_ratings = pivot_table.loc[sampled_users, product_name]  # 겹치는 사용자들의 실제 평점
    predicted_ratings = predicted_ratings_df['예측 평점']  # 예측된 평점
    
    # 모델 평가 (RMSE, MAE)
    rmse, mae = evaluate_model(actual_ratings, predicted_ratings)
    
    # 결과 출력
    print(f"'{product_name}'에 대한 겹치는 사용자 10명의 예측 평점:")
    print(predicted_ratings_df.head(10))  # 상위 10명의 예측된 평점 출력
    
    print(f"\n평가지표:")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
else:
    print(f"겹치는 사용자가 없거나 충분한 평가가 없습니다.")

'[10월 올영픽]  아누아 피디알엔 히알루론산 캡슐 100 세럼 30ml 기획 (+눈물세럼 1ml*10ea)'에 대한 겹치는 사용자 10명의 예측 평점:
            예측 평점
아이디              
근짱헬짱     4.585252
용빈의엄마    4.585252
uo****   4.585252
반짝반짝어린별  4.585252
단이00214  4.585252
동그릉      4.585252
냥이누나     4.585252
뚜까맘      4.585252
gmc5031  3.696189
S2봄봄     3.696189

평가지표:
RMSE: 0.691084270051853
MAE: 0.5925608089429169


In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re

# 크롤링한 데이터를 저장할 리스트
data = []

# 웹드라이버 설정
chrome_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=chrome_service)

# 상품 URL 리스트 (예시)
urls = [
    'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000211001',
    # 더 많은 상품 URL 추가 가능
]

# 각 상품 URL을 순회하며 리뷰와 평점 크롤링
for url in urls:
    driver.get(url)
    time.sleep(3)  # 페이지 로드 대기

    # 리뷰 섹션으로 스크롤
    rv = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a')
    rv.click()
    time.sleep(2)

    # 스크롤을 통해 더 많은 리뷰 로드
    before_h = driver.execute_script("return window.scrollY")
    while True:
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(1)
        after_h = driver.execute_script("return window.scrollY")
        if after_h == before_h:
            break
        before_h = after_h

    # BeautifulSoup을 사용해 HTML 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 리뷰와 평점 크롤링
    reviews = soup.select('.review_cont > p')
    ratings = soup.select('.review_cont > .score_area > .review_point > span')

    for review, rating in zip(reviews, ratings):
        review_text = review.get_text(strip=True)
        rating_value = re.findall(r'\d+', rating.get_text(strip=True))[0]
        data.append([review_text, int(rating_value)])

# DataFrame으로 변환
df = pd.DataFrame(data, columns=['리뷰', '평점'])

# CSV로 저장 (선택 사항)
df.to_csv('review_ratings.csv', index=False)

# 드라이버 종료
driver.quit()

WebDriverException: Message: Service /Users/choiseungmin/.wdm/drivers/chromedriver/mac64/129.0.6668.89/chromedriver-mac-arm64/chromedriver unexpectedly exited. Status code was: -9
